## Assignment-07 First Step of using machine learning and models.

In [75]:
import os
import re
import scipy
import numpy as np
import pandas as pd
from pprint import pprint
from pyltp import Segmentor
from sklearn.svm import SVC
import matplotlib.pyplot as plt
from scipy.spatial.distance import cosine
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier as KNC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

%matplotlib inline

### Step1: 数据分析

请在课程的GitHub中下载数据集，然后使用pandas进行读取。

In [2]:
fname = "D:\\Github\\NLP\\Artificial_Intelligence_for_NLP\\Week_05_0803_pyltp\\sqlResult_1558435.csv"

In [3]:
data = pd.read_csv(fname, encoding='gb18030')

In [4]:
data.head(1)

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm


In [5]:
len(data)

89611

In [6]:
xinhua_news = data[data.source == '新华社']

In [7]:
len(xinhua_news) / len(data)

0.8778051801676133

### 任务描述

报社等相关的机构，往往会遇到一个问题，就是别人家的机构使用自己的文章但是并没有标明来源。 在本次任务中，我们将解决新华社的文章被抄袭引用的问题。

给定的数据集合中，存在一些新闻预料，该预料是来自新华社，但是其来源并不是新华社，请设计技巧学习模型解决该问题。

![](https://timgsa.baidu.com/timg?image&quality=80&size=b9999_10000&sec=1566105348906&di=ee9a2de91207767364853d4decc6cca3&imgtype=0&src=http%3A%2F%2Fmmbiz.qpic.cn%2Fmmbiz_png%2FTicO2kbP6Ao5sCsSQDpehZiczLdC6hDCNvoicjcOCEKX2bLxBc9gVOw28zHyFibfIWq9ceRibP6HDTKReGkr6YyTfQQ%2F640%3Fwx_fmt%3Dpng)

### Step2: 数据预处理

将pandas中的数据，依据是否是新华社的文章，请改变成新的数据dataframe: <content, y>, 其中，content是文章内容，y是0或者1. 你可能要使用到pandas的dataframe操作。[reference](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.apply.html)

In [8]:
label = (data.source == '新华社').astype(int)
content = data.content

In [9]:
data_new = pd.DataFrame({'content':content, 'label':label})

In [10]:
data_new.head()

,content,label
0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


### Step3: 使用tfidf进行文本向量化

#### 新闻内容切词整理 

In [10]:
ltp_model_path = "D:\\Github\\NLP\\Projects\\data\\ltp_data_v3.4.0"
segmentor_model_path = os.path.join(ltp_model_path, 'cws.model')
segmentor = Segmentor()
segmentor.load(segmentor_model_path)

In [11]:
def ltp_cut(document):
    result = ''
    if isinstance(document, str):
        words = segmentor.segment(document)
        words = [re.findall('\d*\w+', x) for x in words]
        result = ' '.join([x[0] for x in words if x and x[0] !='n'])  # pyltp会把换行符\n给分开
        return result if result else None

#### 切词保存数据，这一步还是要花几分钟的

In [16]:
data_new['content_cut'] = data_new.content.apply(lambda i: ltp_cut(i))
data_new.to_csv('cleaned_data.csv')

In [2]:
# index_col 参数不可省略，否则会出现未命名列
df = pd.read_csv('cleaned_data.csv', index_col=0)

In [3]:
# 删除 nan 无数据行
df.dropna(inplace=True)
# 重置索引为连续
df.reset_index(drop=True, inplace=True)

In [4]:
len(df)

86863

In [5]:
df.tail()

,content,label,content_cut
86858,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...
86859,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...
86860,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...
86861,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...
86862,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...


#### 文本向量化

In [19]:
# 指定一定数量的feature，tfidf值小于前该数量的词汇，其tfidf值设置为 0
vectorized = TfidfVectorizer(max_features=200)

In [20]:
X = vectorized.fit_transform(df.content_cut)

In [21]:
X.shape

(86863, 200)

In [66]:
df.label.value_counts()

1    78472
0     8391
Name: label, dtype: int64

* scipy库中consine的实现为：$$cosine(u, v) = 1 - \frac{u \cdot v}{||u||_2 ||v||_2}$$  取值范围[0, 2];因此这个函数的结果越小说明越相似。

### Step4: 参考scikit-learning的方法，构建你的第一个机器学习模型

+ 按照课程讲解的内容，把数据集分割为 traning_data, validation_data, test_data. [reference](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [7]:
X = X.toarray()
y = df.label

In [44]:
y.shape, X.shape

((86863,), (86863, 200))

In [49]:
Xtrain,Xtest, ytrain, ytest = train_test_split(X, y, test_size=.2)
Xtrain, Xcv, ytrain, ycv = train_test_split(Xtrain, ytrain, test_size=.25)
Xtrain.shape,ytrain.shape, Xcv.shape, ycv.shape, Xtest.shape , ytest.shape

((52117, 200), (52117,), (17373, 200), (17373,), (17373, 200), (17373,))

+ 参照scikit learning的示例，构建你的第一个KNN机器学习模型。[reference](https://scikit-learn.org/stable/modules/neighbors.html#neighbors)

### Step5-6: 在traning_data, validation_data, test_data 上观察其相关metric: recall, precision, f1等， 并解释其含义. 

precision:预测正样本的能力。

recall:正确找出正样本的能力。

#### KNN
慢。

In [50]:
clf = KNC()
clf.fit(Xtrain, ytrain)
clf.score(Xcv, ycv)

0.92672537846083

In [51]:
ypred = clf.predict(Xtest)

In [52]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.64      0.54      0.59      1651
           1       0.95      0.97      0.96     15722

    accuracy                           0.93     17373
   macro avg       0.80      0.75      0.77     17373
weighted avg       0.92      0.93      0.92     17373



非新华社文章的判准率很低，即找出非新华社文章的效果不理想，很多非新华社文章被判为新华社文章。

#### LogisticRegression
快。

In [72]:
logi_clf = LogisticRegression(solver='lbfgs')
logi_clf.fit(Xtrain, ytrain)
logi_clf.score(Xcv, ycv), logi_clf.score(Xtest, ytest)

(0.9717953145685835, 0.9709894664134001)

In [70]:
ypred_logi = logi_clf.predict(Xtest)
print(classification_report(ytest, ypred_logi))

              precision    recall  f1-score   support

           0       0.91      0.78      0.84      1651
           1       0.98      0.99      0.98     15722

    accuracy                           0.97     17373
   macro avg       0.94      0.88      0.91     17373
weighted avg       0.97      0.97      0.97     17373



表现中等，作备选，如果KNN表现不如意的话。

### Step7: 调整不同的参数，观察变化

#### 默认参数

In [50]:
clf = KNC()
clf.fit(Xtrain, ytrain)
clf.score(Xcv, ycv)

0.92672537846083

In [51]:
ypred = clf.predict(Xtest)

In [52]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.64      0.54      0.59      1651
           1       0.95      0.97      0.96     15722

    accuracy                           0.93     17373
   macro avg       0.80      0.75      0.77     17373
weighted avg       0.92      0.93      0.92     17373



#### 调整weights 'uniform' --> 'distance'

In [92]:
clf = KNC(weights='distance')
clf.fit(Xtrain, ytrain)
clf.score(Xcv, ycv)

0.9320209520520347

In [93]:
ypred = clf.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.68      0.57      0.62      1651
           1       0.96      0.97      0.96     15722

    accuracy                           0.93     17373
   macro avg       0.82      0.77      0.79     17373
weighted avg       0.93      0.93      0.93     17373



非新华社文章的判准率还是很低，比一般LogisticRegression表现都差。

#### 调整Tfidf的文章向量数，200 --> 800

In [6]:
# 指定一定数量的feature，tfidf值小于前该数量的词汇，其tfidf值设置为 0
vectorized = TfidfVectorizer(max_features=800)
X = vectorized.fit_transform(df.content_cut)
X.shape

(86863, 800)

#### 保存文章向量到df中，供选作部分计算相似度

In [7]:
df['doc_vector'] = pd.DataFrame(X)

In [8]:
df.tail()

,content,label,content_cut,doc_vector
86858,新华社照片，多伦多，2017年6月7日\n（体育）（2）冰球——国家女子冰球队海外选秀在多伦...,1,新华社 照片 多伦多 2017年 6月 7日 体育 2 冰球 国家 女子 冰球队 海外 选秀...,"(0, 53)\t0.17924403286136195\n (0, 48)\t0.1..."
86859,新华社兰州6月3日电（王衡、徐丹）记者从甘肃省交通运输厅获悉，甘肃近日集中开建高速公路、普通...,1,新华社 兰州 6月 3日 电 王衡 徐丹 记者 从 甘肃省 交通 运输厅 获悉 甘肃 近日 ...,"(0, 369)\t0.189599286645538\n (0, 326)\t0.1..."
86860,\n\n2017年5月29日，在法国巴黎郊外的凡尔赛宫，法国总统马克龙出席新闻发布会。（新华...,1,n2017 年 5月 29日 在 法国 巴黎 郊外 的 凡尔赛宫 法国 总统 马克龙 出席 ...,"(0, 548)\t0.05804414816418204\n (0, 34)\t0...."
86861,\n\n2017年5月25日，在美国马萨诸塞州剑桥市，哈佛大学毕业生在毕业典礼上欢呼。（新华...,1,n2017 年 5月 25日 在 美国 马萨诸塞州 剑桥市 哈佛 大学 毕业生 在 毕业 典...,"(0, 519)\t0.06651730113974776\n (0, 48)\t0...."
86862,新华社德国杜塞尔多夫６月６日电题：乒乓女球迷 \n 新华社记者王子江、张寒\n 熊老...,1,新华社 德国 杜塞尔多夫 ６月 ６日 电 题 乒乓 女 球迷 新华社 记者 王子江 张寒 熊...,"(0, 798)\t0.0426538919698091\n (0, 794)\t0...."


In [10]:
X2 = X.toarray()
y = df.label
X2.shape, y.shape

((86863, 800), (86863,))

In [14]:
def corpus_split(X, y):
    Xtrain,Xtest, ytrain, ytest = train_test_split(X, y, test_size=.2)
    Xtrain, Xcv, ytrain, ycv = train_test_split(Xtrain, ytrain, test_size=.25)
    return Xtrain,ytrain, Xcv, ycv, Xtest , ytest

In [15]:
Xtrain, ytrain, Xcv, ycv, Xtest, ytest = corpus_split(X2, y)
Xtrain.shape,ytrain.shape, Xcv.shape, ycv.shape, Xtest.shape , ytest.shape

((52117, 800), (52117,), (17373, 800), (17373,), (17373, 800), (17373,))

In [15]:
clf = KNC(weights='distance')
clf.fit(Xtrain, ytrain)
clf.score(Xcv, ycv)

0.9118747481724515

In [16]:
ypred = clf.predict(Xtest)

In [17]:
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.53      0.68      0.60      1699
           1       0.96      0.94      0.95     15674

    accuracy                           0.91     17373
   macro avg       0.75      0.81      0.77     17373
weighted avg       0.92      0.91      0.92     17373



KNN实在是太慢了，打算用Logistics进行参数调优。

判0准确率低的结果是：如果以步骤9为判断依据即判为1但实际为0则为抄袭者，因为很多0被误判为1，而又以预判结果反推来源，将导致很多文章误判为抄袭。

#### LogisticRegression

In [26]:
logi_clf = LogisticRegression(solver='lbfgs')
logi_clf.fit(Xtrain, ytrain)
logi_clf.score(Xcv, ycv), logi_clf.score(Xtest, ytest)

(0.9711045875784263, 0.9707016635008346)

In [27]:
ypred_logi = logi_clf.predict(Xtest)

In [28]:
print(classification_report(ytest, ypred_logi))

              precision    recall  f1-score   support

           0       0.93      0.76      0.84      1699
           1       0.97      0.99      0.98     15674

    accuracy                           0.97     17373
   macro avg       0.95      0.88      0.91     17373
weighted avg       0.97      0.97      0.97     17373



文本向量大小貌似对结果影响不大。

判0的召回率低，意味着当0为positive时，预测结果与实际结果同为0的例数占实际结果的比例偏低，即实际为0，成功预测为0的比例偏低。
召回率低而精确度高，意味着在判为0的结果中，混入少部分被误判为0的1数据，而正确判0的数据，只占实际为0的数据一小部分。

### Step8: 不断改变参数，直到性能达到“某个”点。问：“某个”怎么定义？
训练集的误差会随着训练逐渐减少，验证集的误差会随着模型调整出现先减小，后上升的过程，转折点为模型开始出现过拟合的时刻。

使用GridSearchCV寻找模型最优参数时，数据可以只分train和test，因为传入参数的train会被GridSearchCV分成train和CV两部分进行参数最优寻找。

In [31]:
params = {'C':[.01, .03, .1, .3, 1, 3, 10], 'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']}
clf_logi = LogisticRegression(solver='lbfgs')
clf = GridSearchCV(clf_logi, params, cv=5)
clf.fit(Xtrain, ytrain)

C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs 

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='lbfgs',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.01, 0.03, 0.1, 0.3, 1, 3, 10],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag',
                                    'saga']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [40]:
pd.DataFrame(clf.cv_results_).tail()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
30,12.122041,0.487327,0.019001,0.000633,10,newton-cg,"{'C': 10, 'solver': 'newton-cg'}",0.980622,0.983404,0.984459,0.981579,0.981002,0.982213,0.001474,1
31,4.806591,0.156656,0.017799,0.000749,10,lbfgs,"{'C': 10, 'solver': 'lbfgs'}",0.980622,0.983404,0.984459,0.981483,0.981002,0.982194,0.001483,5
32,1.363001,0.013709,0.017002,0.000631,10,liblinear,"{'C': 10, 'solver': 'liblinear'}",0.980526,0.983404,0.984459,0.981675,0.981002,0.982213,0.001488,1
33,14.128601,1.809070,0.016202,0.000399,10,sag,"{'C': 10, 'solver': 'sag'}",0.980526,0.983404,0.984459,0.981675,0.981002,0.982213,0.001488,1
34,22.189801,0.604848,0.016202,0.000746,10,saga,"{'C': 10, 'solver': 'saga'}",0.980622,0.983404,0.984459,0.981579,0.981002,0.982213,0.001474,1


In [35]:
clf.best_params_, clf.best_score_

({'C': 10, 'solver': 'newton-cg'}, 0.9822130974538058)

In [16]:
logi_clf = LogisticRegression(C=10, solver='newton-cg')
logi_clf.fit(Xtrain, ytrain)
logi_clf.score(Xcv, ycv), logi_clf.score(Xtest, ytest)

(0.9827893858285845, 0.983652794566281)

In [17]:
ypred = logi_clf.predict(Xtest)
print(classification_report(ytest, ypred))

              precision    recall  f1-score   support

           0       0.94      0.89      0.91      1699
           1       0.99      0.99      0.99     15674

    accuracy                           0.98     17373
   macro avg       0.96      0.94      0.95     17373
weighted avg       0.98      0.98      0.98     17373



非新华社文章的判准率还行。

### Step9: 找出所有预测为 1， 但是实际为 0 的文章。 作为抄袭的候选者。
这个标准需要有较高的判0 f1分数支持，否则会出现过多误判的文章。

In [18]:
ypred = logi_clf.predict(X2)
print(classification_report(y, ypred))

              precision    recall  f1-score   support

           0       0.95      0.91      0.93      8391
           1       0.99      0.99      0.99     78472

    accuracy                           0.99     86863
   macro avg       0.97      0.95      0.96     86863
weighted avg       0.99      0.99      0.99     86863



确定用LogisticRegression模型。

In [20]:
y = df.label
y = y.to_numpy()

In [134]:
# 异或找出所有预测和实际不同的结果，再和预测相与，得到的结果中为1的即为预测为1，实际为0的索引
result = (y ^ ypred) & ypred
index = np.ravel(np.argwhere(result))
[(ypred[i], y[i]) for i in index][:10]

[(1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0),
 (1, 0)]

In [25]:
candidates = [df.content[i] for i in index]
candidates_clean = [df.content_cut[i] for i in index]
candidates_vector = [df.doc_vector[i] for i in index]
choice = np.random.choice(len(index))

In [27]:
candidates[choice], candidates_clean[choice], candidates_vector[choice], df.label[choice]

('新华社北京6月19日电（记者 许可）金砖国家外长会晤19日在北京钓鱼台国宾馆举行。外交部长王毅主持会晤，南非国际关系与合作部长马沙巴内、巴西外长努内斯、俄罗斯外长拉夫罗夫、印度外交国务部长辛格出席。\r\n王毅表示，金砖合作机制成立十年来，在五国领导人有力指引下，各方秉持开放、包容、合作、共赢的金砖精神，推动金砖合作从无到有，由浅入深，取得长足发展。五国经济总量占全球比重上升，贸易投资大幅提升，在重要国际金融机构中发言权迈上新层次；合作领域全面拓展，形成全范围、宽领域、多层次的合作架构；在联合国、二十国集团等国际组织中紧密协作，维护广大发展中国家团结和利益，共同应对全球性挑战。\r\n王毅指出，今年是金砖国家第二个十年的开局之年，中国接任金砖国家轮值主席。中方愿同其他四国一道，继续筑牢和充实政治安全、经济金融、人文交流三大合作支柱，积极拓展更多新兴领域合作，推动金砖合作取得更多成果，开启金砖国家第二个“金色十年”。五国应深化务实合作，促进共同发展；加强全球治理，共同应对挑战；开展人文交流，夯实民意基础；推进机制建设，构建更广泛伙伴关系。\r\n王毅强调，金砖国家领导人第九次会晤将于9月在厦门举行，这是一次承前启后、继往开来的重要会晤，将对金砖合作的未来发展产生深远影响。中方愿同其他四国一道，积极开展相关筹备工作。\r\n马沙巴内、努内斯、拉夫罗夫、辛格表示，金砖国家合作强劲有力，取得重要成就。五国应继续秉持金砖精神，推动五国间合作深入发展，在全球事务中发挥更重要作用。四国高度评价中方作为金砖国家轮值主席国所做工作，承诺全力支持、配合中方筹备好领导人第九次会晤，确保会晤取得圆满成功。\r\n五国外长回顾总结了过去十年五国合作的成果和经验，就金砖合作下一步发展、当前国际形势和热点问题进行深入探讨。各方一致认为，金砖国家应推动新兴市场国家及发展中国家团结合作，维护和增进广大发展中国家整体利益和福祉；继续致力于维护国际公平正义、促进世界和平稳定，维护联合国在国际事务中的核心地位，推动政治解决热点问题，携手应对全球性挑战，努力构建合作共赢的新型国际关系；促进世界多极化和国际关系民主化，推动国际秩序和国际体系朝着更加公平合理方向发展，为人类社会集体繁荣进步贡献更多“金砖智慧”和“金砖方案”，为构建人类命运共同体发挥积极和建设性作用。\r\n会晤后，五国外长共同会见记者。\r

### Step10： 总结该过程，什么是数据思维？什么是机器学习思维？
数据思维：给定模型，如何通过调整数据结构得到最优结果；

机器学习思维：给定数据输入，如果通过调整参数和超参数得到最优结果。

### (Optional)使用第4课讲解的 edit distance，在涉嫌抄袭的文章中，找到其重复的文字与被修改过的文字。 

#### 找出和候选文章最像的新华社文章

In [53]:
copy = candidates_vector[choice].toarray()
xinhua_news = df.doc_vector[df.label==1]

In [38]:
xinhua_news

3          (0, 691)\t0.190178084097007\n  (0, 463)\t0.1...
39         (0, 691)\t0.190178084097007\n  (0, 463)\t0.1...
95         (0, 350)\t0.12793199203520353\n  (0, 348)\t0...
111        (0, 666)\t0.10523748778563713\n  (0, 634)\t0...
303        (0, 712)\t0.11637258658237533\n  (0, 419)\t0...
                               ...                        
86858      (0, 53)\t0.17924403286136195\n  (0, 48)\t0.1...
86859      (0, 369)\t0.189599286645538\n  (0, 326)\t0.1...
86860      (0, 548)\t0.05804414816418204\n  (0, 34)\t0....
86861      (0, 519)\t0.06651730113974776\n  (0, 48)\t0....
86862      (0, 798)\t0.0426538919698091\n  (0, 794)\t0....
Name: doc_vector, Length: 78472, dtype: object

In [98]:
def get_most_similars(X, candidates_clean, candidates_vector, choice):
    copy = candidates_vector[choice].toarray()
    all_cosine = sorted([(cosine(X[i].toarray(), copy), i) for i in X.index])
    index = all_cosine[0][1]
    pprint(f'疑似抄袭的文章：{candidates_clean[choice]}')
    pprint('--' * 20)
    pprint(f'最可能被抄袭的文章：{df.content_cut[index]}')
    pprint('**' * 50)
    print('\n')
    return index, choice

In [99]:
get_most_similars(xinhua_news, candidates_clean, candidates_vector, 508)

C:\Users\Administrator\Anaconda3\envs\nlp\lib\site-packages\scipy\spatial\distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


('疑似抄袭的文章：原 标题 林郑月娥 将 香港 建设 成 朝气蓬勃 生活 富足 的 城市 新华社 香港 6月 20日 电 题 将 香港 建设 成 朝气蓬勃 '
 '生活 富足 的 城市 访 香港 特别 行政区 候任 行政 长官 林郑月娥 即将 于 7月 1日 宣誓 就职 的 香港 特区 候任 行政 长官 林郑月娥 '
 '认为 香港 回归 祖国 20 年 来 一国两制 的 成功 落实 维护 了 香港 的 繁荣 稳定 与 核心 价值 让 香港 成功 地 应对 了 困难 与 '
 '危机 林 郑月娥 步入 侯任 行政 长官 办公室 准备 接受 采访 新华社 记者 陈晔华 摄 香港 回归 祖国 20 周年 前夕 林 郑月娥 女士 在 '
 '候任 行政 长官 办公室 接受 了 新华社 记者 采访 她 表示 上任 后 将 努力 带领 特区 政府 将 香港 建设 成 国家 有 朝气 的 特别 '
 '行政区 为 市民 提供 满足 感 高 的 生活 与 香港 回归 的 两 次 结缘 1980年 加入 公职 的 林郑月娥 已经 服务 香港 市民 超过 '
 '36年 亲身 经历 并 见证 了 回归 前后 香港 的 发展 与 变迁 她 认为 过去 20 年 香港 落实 一国两制 港人治港 高度 自治 的 方针 '
 '维护 了 香港 的 繁荣 与 稳定 在 中央 的 大力 支持 下 香港 成功 处理 了 困难 与 危机 从而 能够 渡过 难关 焕发 生机 一国两制 的 '
 '落实 是 非常 成功 的 林 郑月娥 说 过去 20 年 尽管 香港 经济 发展 经历 了 起伏 但 经济 总量 累计 增长 达 82 对于 经济 社会 '
 '发展 水平 比较 高 的 经济体 而言 这 是 十分 不 容易 的 在 这 一 过程 中 一国两制 港人治港 高度 自治 的 方针 以及 香港 的 法治 '
 '自由 等 核心 价值 始终如一 这些 都 是 一国两制 的 初心 林 郑月娥 在 接受 记者 采访 新华社 记者 陈晔华 摄 林 郑月娥 曾 于 '
 '1985年 1996年 两 次 参加 中 英 联络 小组 的 工作 第一 次 是 讨论 香港 基本法 第24 条 的 有关 立法 工作 包括 签发 回归 '
 '后 新 的 香港 身份证 以及 保安 入境 等 第二 次 是 参与 编制 1

5304

寻找相似文章的部分需要优化

In [127]:
origin = ''.join(df.content_cut[5304].split())
copy = ''.join(candidates_clean[508].split())

#### 编辑距离

In [105]:
class EditDistance_update:
    def __init__(self, levenshtein=True):
        self.solution = {}
        self.levenshtein = levenshtein
        
    def edit_distance_with_matrix(self, str1, str2):
        """
        @levenshtein: susbstitute cost 2 steps while True, else cost 1 step.
        """
        len_1, len_2 = len(str1)+1, len(str2)+1
        matrix = np.zeros((len_2, len_1), dtype=np.int)
        if str1 and str2:
            matrix[0, :] = np.arange(len_1)
            matrix[:, 0] = np.arange(len_2)
            for i, char1 in enumerate(str2, start=1):
                for j, char2 in enumerate(str1, start=1):
                        
                    left = matrix[i-1, j] + 1
                    top = matrix[i, j-1] + 1
                    if char1 != char2:
                        diag = (matrix[i-1, j-1] + 2) if self.levenshtein else (matrix[i-1, j-1] + 1)
                    else:
                        diag = matrix[i-1, j-1]
                    
                    matrix[i, j] = min(left, top, diag)

            return matrix[len_2-1 , len_1-1], matrix
        else:
            return (len_1,matrix)  if str1 else (len_2,matrix)
        
    def get_matrix_solution(self, matrix):
        """
        Up refers to inserting; left refers to deleting; digonal refers to substituting.
        """
        r, c = matrix.shape
        if r < 2:
            for _ in matrix[0, :c-1]:
                self.solution[_+1] = f"delete str1.({_})"
            return
        elif c < 2:
            for _ in matrix[:r-1, 0]:
                self.solution[_ + 1] = f"insert before str1.({0}) with str2.({_})"
            return

        row, column = np.array(matrix.shape) - 1
        target = matrix[row, column]
        up = row - 1, column
        left = row, column - 1
        diag = row - 1, column - 1

        temp = min(matrix[diag], matrix[up], matrix[left])
        
        flag = 2 if self.levenshtein else 1

        if target != temp:
            # 删除或者插入优先
            if target == matrix[up] + 1:
                self.solution[target] = f"insert after str1.({column - 1}) with str2.({row - 1})"
                return self.get_matrix_solution(matrix[:row, :])
            elif target == matrix[left] + 1:
                self.solution[target] = f"delete str1.({column - 1})"
                return self.get_matrix_solution(matrix[:, :column])
            elif target == matrix[diag] + flag:
                self.solution[target] = f"substitute str1.({column - 1}) with str2.({row - 1})"
                return self.get_matrix_solution(matrix[:row,:column])
        return self.get_matrix_solution(matrix[:row,:column])
    
    def get_solution(self):
        return sorted(self.solution.items())

In [128]:
test_update = EditDistance_update()
step, matrix_ = test_update.edit_distance_with_matrix(origin, copy)
test_update.get_matrix_solution(matrix_)
test_update.get_solution()

[(1, 'insert before str1.(0) with str2.(0)'),
 (2, 'insert before str1.(0) with str2.(1)'),
 (3, 'insert before str1.(0) with str2.(2)'),
 (4, 'insert before str1.(0) with str2.(3)'),
 (5, 'insert before str1.(0) with str2.(4)'),
 (6, 'insert before str1.(0) with str2.(5)'),
 (7, 'insert before str1.(0) with str2.(6)'),
 (8, 'insert before str1.(0) with str2.(7)'),
 (9, 'insert before str1.(0) with str2.(8)'),
 (10, 'insert before str1.(0) with str2.(9)'),
 (11, 'insert before str1.(0) with str2.(10)'),
 (12, 'insert before str1.(0) with str2.(11)'),
 (13, 'insert before str1.(0) with str2.(12)'),
 (14, 'insert before str1.(0) with str2.(13)'),
 (15, 'insert before str1.(0) with str2.(14)'),
 (16, 'insert before str1.(0) with str2.(15)'),
 (17, 'insert before str1.(0) with str2.(16)'),
 (18, 'insert before str1.(0) with str2.(17)'),
 (19, 'insert before str1.(0) with str2.(18)'),
 (20, 'insert before str1.(0) with str2.(19)'),
 (21, 'insert before str1.(0) with str2.(20)'),
 (22, 'inse

In [133]:
len(origin), len(copy)

(2065, 2229)

两千字出头的文章，更改两百个字就成为另一篇文章，重复度还是很高的。

### Step11: 利用第8课讲述的新模型，进行操作，感受其中不同的参数、模型对性能的影响。
见第八课assignment.